# Milestone 3 Assignment

Author: Ajai Shukla
Create a new notebook and perform each of the following tasks and answer the related questions:

    1). Build a simple neural networks model
    2). Build a DNN model
    3). Build a RNN model
    Summarize your findings with examples.  Explain what the manufacturer should focus on to optimize the diaper manufacturing process.


In [422]:
!pip install pandas

In [354]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [355]:
import os
import numpy as np
import time
import datetime
import pandas as pd
import tensorflow as tf
from numpy import loadtxt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Masking
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv3D

In [356]:
print(tf.__version__)

2.0.0-alpha0


In [357]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [358]:
def data(url, filename ):
    
    
    ## Read the .csv file with the pandas read_csv method
    data_url = pd.read_csv(url, sep = " ", header=None)
    data_url.to_csv(filename,sep=",", index=False)
    file_path = x=os.getcwd()
    #print file_path
    file_name = find(filename, file_path)
    #print file_name
    data = pd.read_csv(file_name)
    nRow, nCol = data.shape
    print(f'There are {nRow} rows and {nCol} columns')
    
    return data

In [359]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
filename = 'secom.csv'
secom_data = data(url, filename)

There are 1567 rows and 590 columns


In [360]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'
filename = 'labels.csv'
labels = data(url, filename)

There are 1567 rows and 2 columns


In [361]:
labels = labels.rename(columns={"0": "classification", "1": "timestamp"})
dataset = pd.concat([secom_data, labels['classification']], axis=1)
#secom = dataset[['54','59', '162', '348', '172', '427', '134', '420', '320', 'classification']]

In [362]:
#secom['classification'] = secom['classification'].replace([-1], 0)
dataset['classification'] = dataset['classification'].replace([-1], 0)

In [363]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,581,582,583,584,585,586,587,588,589,classification
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,0
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,0
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,0
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,0


In [364]:
dataset.isnull().any().any()
# Replacing all the NaN values with 0 as the values correspond to the test results.
dataset = dataset.replace(np.NaN, 0)
# again, checking if there is any NULL values left
dataset.isnull().any().any()

False

In [365]:
# split into input (X) and output (y) variables
x = dataset.drop(columns=['classification'], axis=1)
y = dataset['classification']

Since our input features are at different scales we need to standardize the input.

In [366]:
sc = StandardScaler()
x = sc.fit_transform(x)
x

array([[ 0.13999828,  0.42920791,  0.03273532, ..., -1.86846064,
        -1.84076867, -1.0611589 ],
       [ 0.46402024, -0.10587396,  0.23685169, ...,  0.41258698,
         0.25102908,  1.15695081],
       [-0.35125598,  0.40723307,  0.02641324, ...,  3.62421124,
         3.31899911, -0.17840653],
       ...,
       [-0.12041844, -0.5678868 ,  0.12151173, ..., -0.89249002,
        -0.96918628, -0.59748491],
       [-0.53957345,  0.25606135, -0.01842658, ...,  0.91192079,
         0.77397852, -0.06511812],
       [-0.28974927, -0.18370602,  0.06960583, ..., -0.03000435,
        -0.27192036,  0.40672924]])

In [367]:
Counter(y)

Counter({0: 1463, 1: 104})

We now split the input features and target variables into training dataset and test dataset. out test dataset will be 30% of our entire dataset

In [368]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

# gettiing the shapes
print("shape of x_train: ", x_train.shape)
print("shape of x_test: ", x_test.shape)
print("shape of y_train: ", y_train.shape)
print("shape of y_test: ", y_test.shape)

shape of x_train:  (1096, 590)
shape of x_test:  (471, 590)
shape of y_train:  (1096,)
shape of y_test:  (471,)


In [369]:
x.shape, y.shape

x_train.shape, y_train.shape


((1096, 590), (1096,))

In [370]:
print(x_train.shape)
print(x_train.shape[0:])

(1096, 590)
(1096, 590)


We have preprocessed the data and we are now ready to build the neural network.
We are using keras to build our neural network. We import the keras library to create the neural network layers.
There are two main types of models available in keras — Sequential and Model. we will use Sequential model to build our neural network.

## 1). Build a simple neural networks model

In [371]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim=590))
#Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [372]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [373]:
#Fitting the data to the training dataset
classifier.fit(x_train,y_train, batch_size=64, epochs=20)

Epoch 1/20
1096/1096 [==============================] - 0s 254us/sample - loss: 0.6895 - accuracy: 0.8951
Epoch 2/20
1096/1096 [==============================] - 0s 25us/sample - loss: 0.6793 - accuracy: 0.9288
Epoch 3/20
1096/1096 [==============================] - 0s 22us/sample - loss: 0.6644 - accuracy: 0.9288
Epoch 4/20
1096/1096 [==============================] - 0s 25us/sample - loss: 0.6388 - accuracy: 0.9288
Epoch 5/20
1096/1096 [==============================] - 0s 22us/sample - loss: 0.5934 - accuracy: 0.9288
Epoch 6/20
1096/1096 [==============================] - 0s 24us/sample - loss: 0.5197 - accuracy: 0.9288
Epoch 7/20
1096/1096 [==============================] - 0s 28us/sample - loss: 0.4320 - accuracy: 0.9288
Epoch 8/20
1096/1096 [==============================] - 0s 23us/sample - loss: 0.3521 - accuracy: 0.9288
Epoch 9/20
1096/1096 [==============================] - 0s 28us/sample - loss: 0.2931 - accuracy: 0.9288
Epoch 10/20
1096/1096 [==============================]

We can also evaluate the loss value & metrics values for the model in test mode using evaluate function

In [374]:
eval_model=classifier.evaluate(x_train, y_train)
eval_model

1096/1096 [==============================] - 0s 59us/sample - loss: 0.1610 - accuracy: 0.9288


[0.16096634760390233, 0.9288321]

We now predict the output for our test dataset. If the prediction is greater than 0.5 then the output is 1 else the output is 0

In [375]:
y_pred=classifier.predict(x_test)
y_pred =(y_pred>0.5)

Now is the moment of truth. we check the accuracy on the test dataset

In [376]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[445   0]
 [ 26   0]]


## 2). Build a DNN model
we will now build our model which requires us to configure its layers, and then compile it. For this problem we will have 3 layers: an input layer, a hidden layer, and output layer. We will build a dense network, where all the neurons are connected to the next layer

In [377]:
from keras.models import Sequential
from keras.layers import Dense

In [378]:
#create model
model = keras.Sequential()

In [379]:
n_cols = x_train.shape[1]

In [380]:
#add model layers
model.add(keras.layers.Flatten(input_shape =(n_cols,)))
model.add(keras.layers.Dense(512, activation='relu', input_shape=(n_cols,)))
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(1))

In [381]:
keras.layers.Conv3D

tensorflow.python.keras.layers.convolutional.Conv3D

In [382]:
#compile model using mse as a measure of model performance
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])

In [383]:
#train model
model.fit(x_train, y_train, validation_split=0.2, epochs=30)

Train on 876 samples, validate on 220 samples
Epoch 1/30
876/876 [==============================] - 0s 385us/sample - loss: 0.4459 - accuracy: 0.8265 - val_loss: 0.0926 - val_accuracy: 0.8955
Epoch 2/30
876/876 [==============================] - 0s 161us/sample - loss: 0.1597 - accuracy: 0.9224 - val_loss: 0.0923 - val_accuracy: 0.9227
Epoch 3/30
876/876 [==============================] - 0s 148us/sample - loss: 0.0770 - accuracy: 0.9304 - val_loss: 0.0992 - val_accuracy: 0.9227
Epoch 4/30
876/876 [==============================] - 0s 149us/sample - loss: 0.0587 - accuracy: 0.9452 - val_loss: 0.1142 - val_accuracy: 0.9182
Epoch 5/30
876/876 [==============================] - 0s 145us/sample - loss: 0.1272 - accuracy: 0.9555 - val_loss: 0.0779 - val_accuracy: 0.9227
Epoch 6/30
876/876 [==============================] - 0s 148us/sample - loss: 0.0812 - accuracy: 0.9532 - val_loss: 0.1062 - val_accuracy: 0.9227
Epoch 7/30
876/876 [==============================] - 0s 155us/sample - loss: 

In [384]:
eval_model=model.evaluate(x_train, y_train)
eval_model

1096/1096 [==============================] - 0s 46us/sample - loss: 0.0149 - accuracy: 0.9845


[0.014888007728374236, 0.984489]

In [385]:
y_pred=model.predict(x_test)
y_pred =(y_pred>0.5)

Now is the moment of truth. we check the accuracy on the test dataset

In [386]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[444   1]
 [ 25   1]]


## 3). Build a RNN Model

Compute the output with the forward step:
The forward step will unroll the network, and compute the forward activations just as in regular backpropagation. The final output will be used to compute the loss function from which the error signal used for training will be derived. 

In [387]:
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
import tensorflow as tf
from tensorflow import keras

In [388]:
x_train = x_train/255.0
x_test = x_test/255.0

print(x_train.shape)
print(x_train[0].shape)

(1096, 590)
(590,)


In [393]:
x_train = np.reshape(x_train, (x_train.shape[0], 1,x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

In [398]:
batch_size = 64

input_dim = 590

units = 128
output_size = 128  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
  # CuDNN is only available at the layer level, and not at the cell level.
  # This means `LSTM(units)` will use the CuDNN kernel,
  # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
  if allow_cudnn_kernel:
    # The LSTM layer with default options uses CuDNN.
    lstm_layer = tf.keras.layers.LSTM(units, input_shape=(None, input_dim))
  else:
    # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
    lstm_layer = tf.keras.layers.RNN(
        tf.keras.layers.LSTMCell(units),
        input_shape=(None, input_dim))
  model = tf.keras.models.Sequential([
      lstm_layer,
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(output_size, activation='softmax')]
  )
  return model

In [399]:
model = build_model(allow_cudnn_kernel=True)

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='sgd',
              metrics=['accuracy'])

In [400]:
model.summary()

Model: "sequential_121"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm_126 (UnifiedLST (None, 128)               368128    
_________________________________________________________________
batch_normalization_v2_12 (B (None, 128)               512       
_________________________________________________________________
dense_106 (Dense)            (None, 128)               16512     
Total params: 385,152
Trainable params: 384,896
Non-trainable params: 256
_________________________________________________________________


In [401]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=batch_size,
          epochs=5)

Train on 1096 samples, validate on 471 samples
Epoch 1/5
1096/1096 [==============================] - 1s 558us/sample - loss: 4.8379 - accuracy: 0.0611 - val_loss: 4.8177 - val_accuracy: 0.9448
Epoch 2/5
1096/1096 [==============================] - 0s 80us/sample - loss: 4.8041 - accuracy: 0.3896 - val_loss: 4.7836 - val_accuracy: 0.9448
Epoch 3/5
1096/1096 [==============================] - 0s 86us/sample - loss: 4.7705 - accuracy: 0.7400 - val_loss: 4.7495 - val_accuracy: 0.9448
Epoch 4/5
1096/1096 [==============================] - 0s 84us/sample - loss: 4.7369 - accuracy: 0.8604 - val_loss: 4.7151 - val_accuracy: 0.9448
Epoch 5/5
1096/1096 [==============================] - 0s 80us/sample - loss: 4.7030 - accuracy: 0.8987 - val_loss: 4.6809 - val_accuracy: 0.9448


In [402]:
eval_model=model.evaluate(x_train, y_train)
eval_model

1096/1096 [==============================] - 0s 46us/sample - loss: 4.6835 - accuracy: 0.9288


[4.683455905775084, 0.9288321]

Summarize your findings with examples.  Explain what the manufacturer should focus on to optimize the diaper manufacturing process.

we used Simple neuro network model, DNN model and RNN model to analye SECOM dataset, as per SNN model loss was around 15 % and accuracy was 93 % . and with DNN model loss was 1.6 % and accuracy was 98 % that was showing overfiting.
RNN model was not bad and this improved overfitting over other model loss was 4 % and accuracy was 92 %